# Agent Registration Validation

This notebook pings an agent API URL and then registers it in the registry.
Update `APP_HOST` and `API_URL` before running.

In [1]:
from __future__ import annotations

import httpx
from databricks.sdk import WorkspaceClient

APP_HOST = "localhost:8000"
API_URL = "https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints/mas-45a56b70-endpoint/invocations"

if APP_HOST.startswith("http://") or APP_HOST.startswith("https://"):
    BASE_URL = APP_HOST.rstrip("/")
else:
    BASE_URL = f"http://{APP_HOST}".rstrip("/")

headers = WorkspaceClient().config.authenticate()

print("BASE_URL:", BASE_URL)
print("API_URL:", API_URL)

BASE_URL: http://localhost:8000
API_URL: https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints/mas-45a56b70-endpoint/invocations


In [2]:
def ping_api_url() -> None:
    payload = {"input": "ping", "metadata": {"source": "notebook"}}
    resp = httpx.post(API_URL, json=payload, headers=headers, timeout=20.0)
    resp.raise_for_status()
    print("Ping status:", resp.status_code)

ping_api_url()

Ping status: 200


In [ ]:
def register_agent() -> dict:
    payload = {
        "agent_id": "mas-45a56b70-endpoint",
        "version": "v1",
        "api_url": API_URL,
        "card": {
            "name": "mas-45a56b70-endpoint",
            "description": "Databricks agent endpoint registered via notebook.",
            "url": "/a2a",
            "version": "1.0.0",
            "defaultInputModes": ["text"],
            "defaultOutputModes": ["text"],
            "capabilities": {"streaming": False},
            "skills": [
                {
                    "id": "mas-45a56b70-endpoint",
                    "name": "mas-45a56b70-endpoint",
                    "description": "Databricks agent endpoint"
                }
            ]
        }
    }
    resp = httpx.post(f"{BASE_URL}/registry/agent-cards", json=payload, headers=headers, timeout=20.0)
    resp.raise_for_status()
    return resp.json()

print(register_agent())

{'status': 'ok', 'agent_id': 'mas-45a56b70-endpoint'}
